## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier(BaseEstimator):
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1
    
    def __init__(self, min_samples_split=2, min_samples_leaf=1, \
                 max_depth=None, sufficient_share=1.0, \
                 criterion='gini', max_features=None):
        self.tree = dict()
        # минимальное число обьектов в узле, необходимое для дальнейшего разделения
        self.min_samples_split = min_samples_split
        # минимальное число объектов в листе
        self.min_samples_leaf = min_samples_leaf
        # максимальная глубина дерева
        self.max_depth = max_depth
        # необходимая доля примеров преобладающего класса для остановки расщепления
        self.sufficient_share = sufficient_share
        # количество классов
        self.num_class = -1
        self.criterion = criterion
        
        # Критерий информативности, по которому считается мера неоднородности. 
        # H(L) - мера неоднородности левой части
        # H(R) - мера неоднородности правой части
        # G_function = left_size/total_size*H(L) + right_size/total_size*H(R) --> min
        # criterion_function = H(L or R)
        if criterion == 'gini':
            self.criterion_function = self.__gini
        elif criterion == 'entropy':
            self.criterion_function = self.__entropy
        elif criterion == 'misclass':
            self.criterion_function = self.__misclass
        else:
            print('invalid criterion name')
            raise
                        
        # Максимальное число признаков, по которым ищется лучшее разбиение в дереве
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise
            
    # left_count_class - число элементов классов в левой части разделенного массива
    # left_sizes - размер левой половины разделенного массива
    # right_count_class - число элементов классов в правой части разделенного массива
    # right_sizes - размер правой половины разделенного массива
    #
    # left_count_class, right_count_class -- shape: число_разбиений x число_классов
    # left_sizes, right_sizes -- shape: число_разбиений x 1
    # Возвращает значения меры неоднородности для каждого разбиения
    # G = left_size/total_size*H(L) + right_size/total_size*H(R) --> min
    def G_function(self, left_count_class, left_sizes, right_count_class, right_sizes):
        imp_left = self.criterion_function(left_count_class/left_sizes).reshape(-1, 1)
        imp_right = self.criterion_function(right_count_class/right_sizes).reshape(-1, 1)

        total_sizes = left_sizes + right_sizes
        return (left_sizes/total_sizes)*imp_left + (right_sizes/total_sizes)*imp_right
    
    @staticmethod
    def __gini(p):
        """p - вероятности классов"""
        return 1 - (p ** 2).sum(axis=1)
    
    @staticmethod
    def __entropy(p):
        return - np.nansum(p * np.log2(p), axis=1)
    
    @staticmethod
    def __misclass(p):
        return 1 - p.max(axis=1)
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.sqrt(n_feature)))
        return feature_ids[:max_features]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        max_features = max(1, int(np.log2(n_feature)))
        return feature_ids[:max_features]

    def __get_feature_ids_N(self, n_feature):
        return list(range(n_feature))
        
    def __sort_samples(self, x, y):
        """
        x - признак, y - целевой переменная.
        Сортировка признака по возрастанию.
        Возвращает отсортированные признак x и соответствующую целевую переменную y.
        """
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]
    
    def __div_samples(self, x, y, feature_id, threshold):
        """Разделение признака feature_id из матрицы признаков x
        и соответствующие значения целевой переменной y
        по пороговому значению threshold:
        если x > threshold, то левое поддерево, 
        если x <= threshold, то правое поддерево.
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    
    def __find_threshold(self, x, y):
        """
        Поиск оптимального разбиения для признака x.
        Возвращает значение функционала G, которое минимизировали, и пороговое значение признака x.
        """
        # Судя по написанному коду в этой функции, автор вместо параметра 
        # min_samples_leaf использовал параметр min_samples_split.
        # Из-за этого были ошибки при построении дерева.
        min_samples = self.min_samples_leaf
        
        # Сортировка x и y по возрастанию x
        sorted_x, sorted_y = self.__sort_samples(x, y)
        # Вычисление количества классов в целевой переменной
        class_number = np.unique(y).shape[0]
        
        # Выборка из отсортированных sorted_y кроме первых и последних min_samples_leaf элементов
        splitted_sorted_y = sorted_y[min_samples:-min_samples]
        # Поиск индексов, в которых меняется значение целевой переменной. 
        # Далее приведение индексов к исходным позициям в матрице признаков.
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + \
                                    (min_samples + 1)
        
        # Если для признака x целевая переменная не изменяется, 
        # то нет смысла делать расщепление
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # В столбцах таблицы class_increments отображается сколько значений класса 
        # появляется с каждым последующим изменением целевой переменной в массиве sorted_y.
        # 
        # Например, первое изменение целевой переменной происходит по индексу idx=r_border_ids[0].
        # Идем с этим индексом idx в исходную отсортированную по признаку x таблицу. 
        # Выше этого индекса idx есть class_increments[0, 0] значений первого класса и 
        # есть class_increments[0, 1] значений второго класса. 
        # Далее происходит второе изменение целевой переменной по индексу idx=r_border_ids[1].
        # В строке class_increments[1, :] хранится сколько еще значений обоих классов добавилось.
        eq_el_count = r_border_ids - np.append([min_samples], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + \
                                np.bincount(sorted_y[:min_samples], minlength=class_number)
        
        # Количество элементов разных классов при различных разбиениях. 
        # l_class_count - верхнее (левое) множество
        # r_class_count - нижнее (правое) множество
        # Пример: i=0 - первое разбиение
        # idx=r_border_ids[i] - индекс первого разбиения
        # l_class_count[i, 0] - количество элементов 1-ого класса в левом множестве после разбиения
        # l_class_count[i, 1] - количество элементов 2-ого класса в левом множестве после разбиения
        # Примечание: элемент sorted_y[idx] считается в правом множестве.
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        
        # Общее число элементов в левом и правом множествах после разбиения
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
        
        # Вычисление функционала G для всех вариантов разбиения признака x
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # Поиск индекса минимального значения G, как наиболее оптимального
        idx = np.argmin(gs)
        
        # Размер левого множества при наилучшем разбиении
        left_el_id = l_sizes[idx][0]
        # Возвращает значение функционала G и само пороговое значение - 
        # среднее значение признака x на границе разделения 
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0


    def __create_leaf_node(self, y, reason_text): 
        labels, samples = np.unique(y, return_counts=True)
        idx = np.argmax(samples)
        top_class_label = labels[idx]
        probs = samples/samples.sum()
        
        return {
            'type': self.LEAF_TYPE, 
            'criterion': self.criterion_function(probs.reshape(1, -1)),
            'class': top_class_label,
            'probability': probs[idx],
            'samples': len(y), 
            'value': samples, 
            'reason': reason_text
        }
    
    def __create_non_leaf_node(self, y, feature, threshold, criterion):
        labels, samples = np.unique(y, return_counts=True)
        idx = np.argmax(samples)
        top_class_label = labels[idx]
        probs = samples/samples.sum()
        
        return {
            'type': self.NON_LEAF_TYPE, 
            'criterion': self.criterion_function(probs.reshape(1, -1)),
            'gvalue': criterion,
            'feature': feature, 
            'threshold': threshold,
            'class': top_class_label,
            'probability': probs[idx],
            'samples': len(y), 
            'value': samples
        }
    
    def __get_top_class(self, y):
        """Возвращает метку преобладающего класса 
        и количество примеров этого класса"""
        labels, samples = np.unique(y, return_counts=True)
        idx = np.argmax(samples)
        return labels[idx], samples[idx]
        
        
    def __fit_node(self, X, y, node_id, depth):
        # Остановка расщепления: достигнута максимальная глубина дерева
        if depth == self.max_depth:
            self.tree[node_id] = self.__create_leaf_node(y, 'max_depth')
            return

        # Остановка расщепления: число элементов в узле меньше необходимого минимума
        if len(y) < 2*self.min_samples_leaf or len(y) < self.min_samples_split:
            self.tree[node_id] = self.__create_leaf_node(y, 'min samples')
            return
        
        # Остановка расщепления: в узле элементы только одного класса
        if len(np.unique(y)) == 1:
            self.tree[node_id] = self.__create_leaf_node(y, 'one class')
            return

        top_class_label, top_class_samples = self.__get_top_class(y)
        # Остановка расщепления: доля элементов преобладающего класса больше заданного порога
        if top_class_samples/len(y) >= self.sufficient_share:
            self.tree[node_id] = self.__create_leaf_node(y, 'sufficient_share')
            return
        
        n_samples, n_features = X.shape

        # Вычисление для заданного множества признаков порогов и значений функционала G
        splits = []
        for feature_id in self.get_feature_ids(n_features):
            gvalue, threshold = self.__find_threshold(X[:, feature_id], y)
            # если для признака x целевая переменная y меняется
            if threshold is not None:
                splits.append((feature_id, threshold, gvalue))

        # Остановка расщепления: недостаточно элементов в узле, расщепление невозможно
        if len(splits) == 0:
            self.tree[node_id] = self.__create_leaf_node(y, 'can\'t split')
            return

        # выбор наилучшего разбиения
        best_split = min(splits, key=lambda x: x[2])
        # разделение данных узла по наилучшему критерию
        x_left, x_right, y_left, y_right = self.__div_samples(X, y, \
                                                              feature_id=best_split[0],
                                                              threshold=best_split[1])
        
        if len(y_left) == 0 or len(y_right) == 0:
            self.tree[node_id] = self.__create_leaf_node(y, \
                                    'left or right node is empty: feature {}, threshold {}'.\
                                        format(best_split[0], best_split[1]))
            return

        # расщепление узла на левое и правое поддеревья
        self.tree[node_id] = self.__create_non_leaf_node(y, \
                                                         feature=best_split[0], \
                                                         threshold=best_split[1], \
                                                         criterion=best_split[2])

        self.__fit_node(x_left, y_left, 2*node_id+1, depth+1)
        self.__fit_node(x_right, y_right, 2*node_id+2, depth+1) 
            
            
    def fit(self, X, y):
        self.num_class = np.unique(y).size
        self.__fit_node(X, y, 0, 0)
        
    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node['type'] == self.__class__.NON_LEAF_TYPE:
            feature_id, threshold = node['feature'], node['threshold']
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node['class']

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node['type'] == self.__class__.NON_LEAF_TYPE:
            feature_id, threshold = node['feature'], node['threshold']
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node['probability']
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
        
    def __print_node(self, node_id, depth, features):
        node = self.tree.get(node_id)
        tabs = '\t'*depth
        
        if node:
            node_type = 'NON_LEAF' if node['type'] == 0 else 'LEAF'
            
            text = ''
            text += '{}type: {}\n'.format(tabs, node_type)
            text += '{}id: {}\n'.format(tabs, node_id)
            text += '{}depth: {}\n'.format(tabs, depth)
            text += '{}{}: {}\n'.format(tabs, self.criterion, node['criterion'][0])
            
            if node_type == 'NON_LEAF':
                feature = features[node['feature']]
                text += '{}gvalue: {}\n'.format(tabs, node['gvalue'][0])
                text += '{}feature: {}\n'.format(tabs, feature)
                text += '{}threshold: {}\n'.format(tabs, node['threshold'])
            
            text += '{}class: {}\n'.format(tabs, node['class'])
            text += '{}probability: {}\n'.format(tabs, node['probability'])
            text += '{}samples: {}\n'.format(tabs, node['samples'])    
            text += '{}value: {}\n'.format(tabs, node['value'])
            
            if node_type == 'LEAF':
                text += '{}reason: {}\n'.format(tabs, node['reason'])
                
            print(text)
            
            self.__print_node(2*node_id+1, depth+1, features)
            self.__print_node(2*node_id+2, depth+1, features)
            
    def print_tree(self, features):
        self.__print_node(0, 0, features)

In [3]:
df = pd.read_csv('cs_training.csv', sep=',').dropna()
df_sample = df.sample(200, random_state=17)
print(df.shape)
df.head()

(120269, 11)


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

X_sample = df_sample.iloc[:, 1:].values
y_sample = df_sample.iloc[:, 0].values

features = df.columns[1:].values

Применим нашу модель на небольшой части датасета и выведем данные построенного дерева:

In [5]:
tree = MyDecisionTreeClassifier(min_samples_split=7, min_samples_leaf=3, sufficient_share=0.95)
tree.fit(X_sample, y_sample)
tree.print_tree(features)

type: NON_LEAF
id: 0
depth: 0
gini: 0.16379999999999995
gvalue: 0.13246073298429303
feature: NumberOfTimes90DaysLate
threshold: 1.0
class: 0
probability: 0.91
samples: 200
value: [182  18]

	type: LEAF
	id: 1
	depth: 1
	gini: 0.0
	class: 1
	probability: 1.0
	samples: 3
	value: [3]
	reason: min samples

	type: NON_LEAF
	id: 2
	depth: 1
	gini: 0.14068901543456414
	gvalue: 0.12595952271079175
	feature: NumberOfTime30-59DaysPastDueNotWorse
	threshold: 1.0
	class: 0
	probability: 0.9238578680203046
	samples: 197
	value: [182  15]

		type: NON_LEAF
		id: 5
		depth: 2
		gini: 0.4012345679012346
		gvalue: 0.2777777777777778
		feature: NumberOfTime30-59DaysPastDueNotWorse
		threshold: 2.0
		class: 0
		probability: 0.7222222222222222
		samples: 18
		value: [13  5]

			type: LEAF
			id: 11
			depth: 3
			gini: 0.5
			class: 0
			probability: 0.5
			samples: 6
			value: [3 3]
			reason: min samples

			type: NON_LEAF
			id: 12
			depth: 3
			gini: 0.2777777777777777
			gvalue: 0.16666666666666666


In [6]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [7]:
t1 = time()
my_clf.fit(X, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(X, y)
t2 = time()
print(t2 - t1)

0.6739053726196289
1.284670114517212


## Проверка качества работы

In [8]:
gkf = KFold(n_splits=5, shuffle=True)

In [9]:
for train, test in gkf.split(X, y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.9341897397522242
0.9289930988608963
0.9328178265569136
0.932776253429783
0.9314846380908827


In [10]:
for train, test in gkf.split(X, y):
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.8893323355782822
0.8922008813502952
0.8904132368836783
0.8945705495967406
0.8913649025069638


# Применить для задачи Titanic 

In [11]:
df_train = pd.read_csv('train.csv', index_col='PassengerId')
df_test = pd.read_csv('test.csv', index_col='PassengerId')

In [12]:
print(df_train.shape)
df_train.head()

(891, 11)


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Минимальная обработка признаков. Посмотрим, как наша модель и дерево из sklearn справятся. 

In [13]:
def prepare_data(df):
    df['Age'].fillna(df['Age'].median(), inplace = True)
    df['Fare'].fillna(df['Fare'].median(), inplace = True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace = True)

    le = LabelEncoder()
    df['Embarked'] = le.fit_transform(df['Embarked'])
    df['Sex'] = le.fit_transform(df['Sex'])
    
    drop_columns = ['Name', 'Ticket', 'Cabin']
    df.drop(drop_columns, axis=1, inplace=True)
    
    return df

In [14]:
df_train = prepare_data(df_train)
df_test = prepare_data(df_test)

In [15]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,,
1,0,3,1,22.0,1,0,7.2500,2
2,1,1,0,38.0,1,0,71.2833,0
3,1,3,0,26.0,0,0,7.9250,2
4,1,1,0,35.0,1,0,53.1000,2
5,0,3,1,35.0,0,0,8.0500,2


In [16]:
df_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
892,3,1,34.5,0,0,7.8292,1
893,3,0,47.0,1,0,7.0000,2
894,2,1,62.0,0,0,9.6875,1
895,3,1,27.0,0,0,8.6625,2
896,3,0,22.0,1,1,12.2875,2


#### Сравнение качества моделей с параметрами по умолчанию на кросс-валидации:

In [17]:
X = df_train.drop(['Survived'], axis=1).values
y = df_train['Survived'].values.ravel()

In [18]:
clf = DecisionTreeClassifier()
my_clf = MyDecisionTreeClassifier()

In [19]:
gkf = KFold(n_splits=7, shuffle=True, random_state=42)

In [20]:
clf_scores = cross_val_score(clf, X, y, cv=gkf, scoring='accuracy').mean()
my_clf_scores = cross_val_score(my_clf, X, y, cv=gkf, scoring='accuracy').mean()

print('sklearn decision tree: {}'.format(clf_scores))
print('my decision tree: {}'.format(my_clf_scores))

sklearn decision tree: 0.7631819460067492
my decision tree: 0.727107353768279


Видно, что наша модель отстает по качеству от дерева из библиотеки sklearn. Посмотрим, какие деревья построили модели.

In [25]:
my_clf = MyDecisionTreeClassifier()
my_clf.fit(X, y)
my_clf.print_tree(features=df_train.columns.values[1:])

type: NON_LEAF
id: 0
depth: 0
gini: 0.4730129578614428
gvalue: 0.33103688673636855
feature: Sex
threshold: 0.0
class: 0
probability: 0.6161616161616161
samples: 891
value: [549 342]

	type: NON_LEAF
	id: 1
	depth: 1
	gini: 0.3064437162277842
	gvalue: 0.28559316448796884
	feature: Age
	threshold: 4.0
	class: 0
	probability: 0.8110918544194108
	samples: 577
	value: [468 109]

		type: NON_LEAF
		id: 3
		depth: 2
		gini: 0.2817709080008861
		gvalue: 0.26024222990947216
		feature: Pclass
		threshold: 1.0
		class: 0
		probability: 0.8303249097472925
		samples: 554
		value: [460  94]

			type: NON_LEAF
			id: 7
			depth: 3
			gini: 0.20740512646265574
			gvalue: 0.2019103870535039
			feature: Age
			threshold: 9.0
			class: 0
			probability: 0.8824884792626728
			samples: 434
			value: [383  51]

				type: LEAF
				id: 15
				depth: 4
				gini: 0.19671695501730113
				class: 0
				probability: 0.8894117647058823
				samples: 425
				value: [378  47]
				reason: left or right node is empty: fe

In [29]:
clf = DecisionTreeClassifier()
clf.fit(X, y)

from sklearn.tree import export_graphviz

def vis_tree(tree_clf, name, feature_names):
    export_graphviz(
        tree_clf,
        out_file=name + ".dot",
        feature_names=feature_names,
        rounded=True,
        filled=True
    )
    
vis_tree(clf, 'titanic_tree', df_train.columns.values[1:])
!dot -Tpng titanic_tree.dot -o titanic_tree.png
!rm titanic_tree.dot 

<img src='titanic_tree.png'>

Реализованная модель построила дерево не настолько глубокое, как у sklearn. Стоит отметить, что деревья до второго уровня совпадают.

#### Подбор параметров

In [22]:
parameters = [
    {
        'min_samples_split': [4],
        'min_samples_leaf': [1, 2], 
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(2, 13)
    }, 
    {
        'min_samples_split': [6],
        'min_samples_leaf': [1, 2, 3], 
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(2, 13)
    }, 
    {
        'min_samples_split': [8],
        'min_samples_leaf': [1, 2, 3, 4], 
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(2, 13)
    }, 
    {
        'min_samples_split': [10],
        'min_samples_leaf': np.arange(1, 6), 
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(2, 13)
    }, 
    {
        'min_samples_split': [12],
        'min_samples_leaf': np.arange(1, 7),
        'criterion': ['gini', 'entropy'],
        'max_depth': np.arange(2, 13)
    }
]

In [34]:
%%time
grid_search = GridSearchCV(MyDecisionTreeClassifier(), parameters, scoring='accuracy', \
                          cv=gkf, n_jobs=-1, verbose=1, iid=False)
grid_search.fit(X, y)

Fitting 7 folds for each of 440 candidates, totalling 3080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 348 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 1848 tasks      | elapsed:   30.1s


CPU times: user 47.2 s, sys: 279 ms, total: 47.5 s
Wall time: 48.3 s


[Parallel(n_jobs=-1)]: Done 3080 out of 3080 | elapsed:   48.3s finished


In [35]:
%%time
grid_search_sklearn = GridSearchCV(DecisionTreeClassifier(), parameters, scoring='accuracy', \
                          cv=gkf, n_jobs=-1, verbose=1, iid=False)
grid_search_sklearn.fit(X, y)

Fitting 7 folds for each of 440 candidates, totalling 3080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 1388 tasks      | elapsed:    1.8s


CPU times: user 1.72 s, sys: 28.6 ms, total: 1.75 s
Wall time: 3.71 s


[Parallel(n_jobs=-1)]: Done 3080 out of 3080 | elapsed:    3.7s finished


In [36]:
print('model: MyDecisionTreeClassifier')
print('Best score:', grid_search.best_score_)
print('Best parameters:\n', grid_search.best_params_)

model: MyDecisionTreeClassifier
Best score: 0.7327140748031497
Best parameters:
 {'criterion': 'gini', 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 4}


In [37]:
print('model: MyDecisionTreeClassifier')
print('Best score:', grid_search_sklearn.best_score_)
print('Best parameters:\n', grid_search_sklearn.best_params_)

model: MyDecisionTreeClassifier
Best score: 0.8282919713160855
Best parameters:
 {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 3, 'min_samples_split': 6}


#### Submit на Kaggle

In [46]:
submission_my_clf = pd.DataFrame({
        "PassengerId": df_test.index.values,
        "Survived": grid_search.best_estimator_.predict(df_test.values)})
submission_my_clf.to_csv('submission_my_clf.csv', index = False)

In [48]:
submission_clf = pd.DataFrame({
        "PassengerId": df_test.index.values,
        "Survived": grid_search_sklearn.best_estimator_.predict(df_test.values)})
submission_clf.to_csv('submission_clf.csv', index = False)

На реализованной модели дерева с простыми признаками и подобранными параметрами сабмит на каггле получился равен 0.77511. С библиотечной моделью дерева с подобранными параметрами получилось 0.76555.